In [1]:
using Graphs
using Plots
using LinearAlgebra
include("../inference/zero_patient.jl")
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear);

In [2]:
q = q_sis
T = 5
N = 10
k = 3
gg = random_regular_graph(N, k)
g = IndexedGraph(gg)
λ = 0.1
κ = 0.08
γ = 3/N
p⁰ = map(1:N) do i
    [1-γ, γ]
end;

In [3]:
function sis_zp(T::Integer, N::Integer, λ::Real, κ::Real, γ::Real,
        nobs::Integer; trunc_dim=2)
    gg = random_regular_graph(N, 3)
    g = IndexedGraph(gg)
    p⁰ = fill([1-γ, γ], N)
    sis = SIS(g, λ, κ, T; p⁰)
    X, bp = simulate_sis(g, λ, κ, p⁰, T, nobs)
    true_zp = findall(isequal(I), X[:,1]) 
    svd_trunc = TruncBond(trunc_dim)
    id_bp, b⁰_bp = find_zero_patients_bp(bp; svd_trunc)
    auc_bp = round(auc(id_bp, true_zp), digits=4)
end

sis_zp (generic function with 1 method)

In [16]:
ProgressMeter.ijulia_behavior(:append)
bondsizes = [2, 4, 6, 8, 10]
aucs_bp = [zeros(0) for _ in bondsizes];

In [ ]:
ninstances = 200
for (i, trunc_dim) in enumerate(bondsizes)
    println("### matrix size: $trunc_dim ###")
    for a in 1:ninstances
        auc_bp = sis_zp(T, N, λ, κ, γ, N; trunc_dim)
        push!(aucs_bp[i], auc_bp)
    end
end

### matrix size: 2 ###


Running MPdBP: iter 2 	 Time: 0:00:00
Running MPdBP: iter 4 	 Time: 0:00:00
Running MPdBP: iter 5 	 Time: 0:00:00
Running MPdBP: iter 7 	 Time: 0:00:00
Running MPdBP: iter 2 	 Time: 0:00:00
Running MPdBP: iter 4 	 Time: 0:00:00
  Δ:  0.02629315750551231

In [ ]:
aucs_bp_avg = mean.(aucs_bp)
aucs_bp_std = std.(aucs_bp) ./ sqrt.(length.(aucs_bp))
println("SIS with: P(S→I)=λ=$λ, P(I→S)=κ=$κ. P(zero patient)=γ=$γ")
println("Random regular N=$N, k=3. T=$T, $N random observations from the prior at final time")
pl = plot(bondsizes, aucs_bp_avg, m=:o, ribbon=aucs_bp_std, label="bp", xlabel="matrix size", 
    ylabel="AUC", legend=:bottomright)